Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

# Head Hunter

In [4]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import time
import re
import random

In [2]:

headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'}


def hh(main_link, search_str, n_str):
    # n_str - кол-во просматриваемых страниц
    html = requests.get(
        main_link + '/search/vacancy?clusters=true&enable_snippets=true&text=' + search_str + '&showClusters=true',
        headers=headers).text
    parsed_html = bs(html, 'lxml')

    jobs = []
    for i in range(n_str):
        jobs_block = parsed_html.find('div', {'class': 'vacancy-serp'})
        jobs_list = jobs_block.findChildren(recursive=False)
        for job in jobs_list:
            job_data = {}
            req = job.find('span', {'class': 'g-user-content'})
            if req != None:
                main_info = req.findChild()
                job_name = main_info.getText()
                job_link = main_info['href']
                salary = job.find('div', {'class': 'vacancy-serp-item__compensation'})

                if not salary:
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    salary = salary.getText().replace(u'\xa0', u'')

                    salary = re.split(r'\s|-', salary)

                    if salary[0] == 'до':
                        salary_min = None
                        salary_max = int(salary[1])
                    elif salary[0] == 'от':
                        salary_min = int(salary[1])
                        salary_max = None
                    else:
                        salary_min = int(salary[0])
                        salary_max = int(salary[1])
                    salary_currency = salary[-1]

                job_data['name'] = job_name
                job_data['salary_min'] = salary_min
                job_data['salary_max'] = salary_max
                job_data['salary_currency'] = salary_currency
                job_data['link'] = job_link
                job_data['site'] = main_link
                jobs.append(job_data)
        time.sleep(random.randint(1, 10))
        next_btn_block = parsed_html.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
        next_btn_link = next_btn_block['href']
        html = requests.get(main_link + next_btn_link, headers=headers).text
        parsed_html = bs(html, 'lxml')

    pprint(jobs)
    return jobs


search_str = 'Data+science'
n_str = 2

hh('https://hh.ru', search_str, n_str)


SyntaxError: unexpected character after line continuation character (<ipython-input-2-08bfbaf44333>, line 34)

In [9]:

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'}

def superjob(main_link, search_str, n_str):
    #n_str - кол-во просматриваемых страниц
    base_url = main_link +'/vacancy/search/?keywords='+ search_str +'&geo%5Bc%5D%5B0%5D=1'
    jobs = []
    session = requests.Session()
    for i in range(n_str):
        request = session.get(base_url, headers = headers)
        if request.status_code == 200:
            soup = bs(request.content, 'lxml')
            divs = soup.find_all('div', {'class':'_3zucV _1fMKr undefined _3tcTB _3SGgo'})
            for div in divs:
                title = div.find('a').getText()
                href = main_link + div.find('a').get('href')
                salary = div.find('span', {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'})
                if salary != None:
                    salary = salary.text
                    salary = salary.replace(u'\xa0', u'')
                    if '—' in salary:
                        salary_min = salary.split('—')[0]
                        salary_min = re.sub(r'[^0-9]', '', salary_min)
                        salary_max = salary.split('—')[1]
                        salary_max = re.sub(r'[^0-9]', '', salary_max)
                        salary_min = int(salary_min)
                        salary_max = int(salary_max)
                    elif 'от' in salary:
                        salary_min = salary[2:]
                        salary_min = re.sub(r'[^0-9]', '', salary_min)
                        salary_min = int(salary_min)
                        salary_max = None
                    elif 'договорённости' in salary:
                        salary_min = None
                        salary_max = None
                    elif 'до' in salary:
                        salary_min = None
                        salary_max = salary[2:]
                        salary_max = re.sub(r'[^0-9]', '', salary_max)
                        salary_max = int(salary_max)
                    else:
                        salary_min = int(re.sub(r'[^0-9]', '', salary))
                        salary_max = int(re.sub(r'[^0-9]', '', salary))
                else:
                    salary_min = None
                    salary_max = None

                jobs.append({
                    'title': title,
                    'href': href,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'link': main_link
                })
            base_url = main_link + \
                       soup.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
            time.sleep(random.randint(1,10))
        else:
            print('Ошибка')

    pprint(jobs)
    return jobs


search_str = 'Python'
n_str = 2




TypeError: 'NoneType' object is not subscriptable